# Managing UETOOLS saves

This section explains how data is written to the UETOOLS save-file and how it can be controlled using UETOOLS. 


## TOC:
* [The variable-specifier YAML files](#yaml-specifiers)
    * [Required variable YAML specifier](#required-yaml)
    * [Optional variable YAML specifier](#optional-yaml)
    * [YAMLs in Case objects](#yamls-in-cases)
* [User-supplied variable YAML specifiers](#user-supplied-yamls)
* [The UETOOLS save-file](#save-files)
* [Using UETOOLS without UEDGE](#uetools-without-uedge)
* [Case.dump](#case-dump)
    


In [1]:
# Move to the folder containing the parameter scans
from os import chdir
from uetools import Case
chdir('testcase_lores')
c=Case("input.yaml")

UEDGE configuration file /Users/holm10/.uetoolsrc read.
UETOOLS-style save successfully restored from /Users/holm10/Documents/fusion/uedge/src/UETOOLS/jupyter/testcase_lores/nc20.hdf5

*** UEDGE arrays populated: Case appears converged ***
fnrm without preconditioning: 1.54e+00



## The variable-specifier YAML files<a class="anchor" id="yaml-specifiers"></a>
The behavior of the UETOOLS save-function is controlled by two external YAML files. One specifies the minimun number of variables required to be saved to have a recoverable UEDGE case from a UETOOLS save file and the other provides the user with the option to specify variables to be stored in the save-files. Below follows a description of each YAML specifier.

### Required variable YAML specifier<a class="anchor" id="required-yaml"></a>
This file containint the minimum number of variables for a case to be recoverable is always read from the UETOOLS module. The YAML file structure is shown below:

In [2]:
!more ../../src/uetools/yamls/requiredvariables.yaml | head -100


# Required variables for UETOOLS to function
# ====================================================
# ======= DO NOT EDIT VARIABLES BELOW ================
# ====================================================

restore:
    bbb:
        - 'tes'
        - 'tis'
        - 'nis'
        - 'ngs'
        - 'tgs' 
        - 'ups'
        - 'phis'
        - 'restart'

grid:
    com:
        - 'nx'
        - 'ny'
        - 'nxm'
        - 'nym'
        - 'ixlb'
        - 'ixpt1'
        - 'ixmdp'
        - 'ixpt2'
        - 'ixrb'
        - 'iysptrx'
        - 'iysptrx1'
        - 'iysptrx2'
        - 'nxleg'
        - 'nxcore'
        - 'nysol'
        - 'nycore'
        - 'rm'
        - 'zm'
        - 'bpol'
        - 'bphi'
        - 'b'
        - 'br'
        - 'vol'
        - 'bz'
        - 'psi'
        - 'nlim'
        - 'xlim'
        - 'ylim'
        - 'rbdry'
        - 'zbdry'
        - 'simagxs'
        - 'sibdrys'
        - 'runid'
        - 'yyrb'
        - 'yylb'
        - 'rmagx

It is, however, useful to consider the resulting Python dictionary, which is used by UETOOLS for defining the variables that are always included in the save-file:

In [3]:
required = c.tools.readyaml("../../src/uetools/yamls/requiredvariables.yaml")
print("requiredvariables.yaml contains the following variable decks:")
print(60*"=")
for deck in required.keys():
    print("-", deck)

requiredvariables.yaml contains the following variable decks:
- restore
- grid
- diffusivities
- setup
- species


Each of the variable "decks" contains the UEDGE packages ot the variables as subdictionaries according to the specifier YAML file, and terminates at the third level with a list of variables in that package to be saved:

In [4]:
print("<restore>")
for packagekey, package in required["restore"].items():
    print("    ",packagekey)
    for variable in package:
        print("        -", variable)

<restore>
     bbb
        - tes
        - tis
        - nis
        - ngs
        - tgs
        - ups
        - phis
        - restart


Each of the variable decks specify the required variables for a specific part of the code:
 - restore: contains the state-variables, used to restore the state of the equation
 
 - grid: contains the data required to construc the computational mesh
    - Some additional data is stored to ensure plotting routines will work and that data from the EFIT files are not lost when reading grids from a file
    
- diffusivities: contains the diffusivity profiles, in case they are used
    - All of the variables are stored regardless of the model used in each case
    - For more information, see Chapter 7
    
- setup: a small set of miscellaneous variables that defines the case setup

All of the above data is automatically read from the appropriate YAML file supplied by the UETOOLS module, and stored to UETOOLS save-files.

### Optional variable YAML specifier<a class="anchor" id="optional-yaml"></a>
Additional to be written to the UETOOLS save-files can be specified/controlled using an external YAML file. In case no user-specified YAML file is defined when creating a Case-object, a default YAML files is used to specify additional data to be written to the save file:

In [5]:
!more ../../uetools/yamls/standardvariables.yaml | head -n 150

../../uetools/yamls/standardvariables.yaml: No such file or directory


The optinal variable YAML specifier has the same structure as the required variable YAML specifier: the first level specifies variable "decks", which contain the UEDGE packages where the variables are located, and the third level contains a list of variables to be written to the save file. There are no restrictions on the number of variable decks or variables within the decks that are written to the save-file. 

The Python structure of the above file is the following:

In [6]:
optional = c.tools.readyaml("../../src/uetools/yamls/standardvariables.yaml")
print("satndardvariables.yaml contains the following variable decks:")
print(60*"=")
for deck in optional.keys():
    print("-", deck)

satndardvariables.yaml contains the following variable decks:
- constants
- centered
- staggered
- boundary
- postprocessed


The variables specified in each of the above decks are variables that are used by various internal UETOOLS functions, which is why they have been included. Removing any of the variables may break certain UETOOLS functionalities, but should not affect the ability to restore cases from YAML inputs/UTEOOLS save files.

### YAMLs in Case objects<a class="anchor" id="yamls-in-cases"></a>
The two YAML specifiers are concatenated and stored to the UETOOLS Case object, and the dictionary is accessed when Case.save is called. Case.save iterates the dictionary and stores every variable in the save-file using the structure specified by the save-files.

In [7]:
print("The Case object contains the following variable decks:")
print(60*"=")
for deck in c.variables['input'].keys():
    print("-", deck)

The Case object contains the following variable decks:
- restore
- grid
- diffusivities
- setup
- species
- constants
- centered
- staggered
- boundary
- postprocessed


## User-supplied variable YAML specifiers<a class="anchor" id="user-supplied-yamls"></a>
As mentioned above, the optional YAML specifier can be a user-specified YAML file, rather than the standard YAML file. The optional YAML specifier can be specified either by configuring the .uetoolsrc file using Case.CreatConfig function (see Chapter 1), editing the .uetoolsrc-file manually, or by supplying the kwarg "variableyamlfile" when a Case object is created. 


    
The kwarg "variableyamlfile" takes precedence over any other optional variable YAML specifiers. If the kwarg is not used when a Case object is created, UETOOLS will look for the "variableyamlfile" in the .uetoolsrc YAML file. The "variableyamlfile" can be added to .uetoolsrc (located in the user HOME-directory) by re-running the Case.CreateConfig function and following the prompts. Alternatively, the "variableyamlfile" argument can be added manually by editing the .uetoolsrc-file (or creating it in the user HOME-directory, if it does not exist) and adding the following line:

    variableyamlfile: <path to optional variable YAML specifier>
    
**NOTE:** as mentioned above, removing variables compared to the default YAML file may result in some functionalities of UETOOLS no longer working. 

**NOTE:** when implementing functionalities to UETOOLS that require additional variables to be available from save-files that are added to a user's optional variable YAML specifiers, other users' YAML specifiers may not contain the additional data, causing the funcitonality to fail. Thus, it is recommended to update the UETOOLS default optional variable YAML specifier in pull-requests that add capabilities to UETOOLS, which assumes some data to be available in the save-files.

## The UETOOLS save-file<a class="anchor" id="save-files"></a>
Now that YAML specifiers are understood, it is time to look at the data written to UETOOLS save-files by Case.save:

In [8]:
c.tools.hdf5tree("nc20.hdf5", depth=1)

<HDF5 file "nc20.hdf5" (mode r)>
├── boundary
├── centered
├── constants
├── diffusivities
├── grid
├── postprocessed
├── restore
├── setup
├── species
└── staggered


As can be seen, all the variable decks specified by the two YAML specifiers are also present in the save-file. Additionally, the structure defined by the YAML specifier is preserved throughout:

In [9]:
c.tools.hdf5tree("nc20.hdf5", "restore")

<HDF5 file "nc20.hdf5" (mode r)>
restore
└── bbb
    ├── ngs (18, 10, 2)
    ├── nis (18, 10, 8)
    ├── phis (18, 10)
    ├── restart (int)
    ├── tes (18, 10)
    ├── tgs (18, 10, 2)
    ├── tis (18, 10)
    └── ups (18, 10, 8)


There is, however, a group written to the save-file, which is **not** included in the YAML specifiers, namely the "setup" group:

In [10]:
c.tools.hdf5tree("nc20.hdf5", "setup", depth=1)

<HDF5 file "nc20.hdf5" (mode r)>
setup
├── atoms
├── boundaryconditions
├── carbon
├── casename (bytes)
├── detected
├── differencing
├── diffusivities
├── fluxlim
├── grid
├── physics
├── rates
├── recycling
├── restart (int)
├── savefile (bytes)
├── solver
└── species


The setup group, as seen above, contains the input decks specified in the input file, along with any subdecks and vairables specified in each:

In [11]:
c.tools.hdf5tree("nc20.hdf5", "setup/atoms")

<HDF5 file "nc20.hdf5" (mode r)>
setup/atoms
├── cfnidh (float)
├── ineudif (int)
├── isngon (6)
├── isupgon (6)
└── ziin (31)


Thus, the save-file contains a virtual copy of the UETOOLS input file used to define the case. Any changes to any of the variables defined in the input file will also be updated as they are changed and a save-file written.

The special group "setup/detected" is also added to all save-files to keep track of changes to input variables which are **not** specified in the UETOOLS YAML input file if changed:

In [12]:
c.tools.hdf5tree("nc20.hdf5", "setup/detected")

<HDF5 file "nc20.hdf5" (mode r)>
setup/detected
├── albedoi (18, 2)
├── albedoo (18, 2)
├── alblb (10, 6, 2)
├── albrb (10, 6, 2)
├── alfe (8)
├── betai (8)
├── chgstate_format (7)
├── coll_fe (18, 10)
├── coll_fi (18, 10)
├── dx0 (float)
├── effvng (2)
├── ennorm (float)
├── flalfea (18)
├── flalfgxa (18, 10)
├── flalfgxya (18, 10)
├── flalfgya (10, 10)
├── flalfia (18)
├── flalftgxa (18)
├── flalftgxya (18)
├── flalftgya (10)
├── flalfva (18)
├── flalfvgxa (18)
├── flalfvgxya (18)
├── flalfvgya (10)
├── floor_cons (13)
├── fnorm (2)
├── iesori (10)
├── iesoro (10)
├── iscolnorm (int)
├── isflxvar (int)
├── isrscalf (int)
├── issori (10)
├── issoro (10)
├── isyloext (int)
├── ivolcur (8)
├── ivolcurg (2)
├── ix_ejcsor (int)
├── ix_sjcsor (int)
├── iy_ejcsor (int)
├── iy_sjcsor (int)
├── jcvsor (float)
├── lynix (2, 18, 8)
├── lyphix (2, 18)
├── lytex (2, 18)
├── lytix (2, 18)
├── mvolcur (8)
├── n0 (31)
├── n0g (6)
├── nfsp (int)
├── nibprof (10)
├── nnorm (float)
├── norm_cons (13)
├

As can be seen, these are the same variables for which changes are detected for, as discussed in Chapter 4. It is this automatical tracking and storing of changes to input variables that allows UETOOLS to use its save-files as standalone setups for UEDGE cases.

## Using UETOOLS without UEDGE<a class="anchor" id="uetools-without-uedge"></a>
Because UETOOLS saves all the data required by UETOOLS functionalities, this means that UETOOLS can be used to process and investigate UEDGE saves without starting UEDGE. In fact, all UETOOLS functionalities can be used without having UEDGE installed, provided Case objects are created using the inplace=True option (Chapter 2), or the data is analyzed using Database-objects (Chapter 4). 

## Case.dump<a class="anchor" id="case-dump"></a>
There are, however, situations when a a complete UEDGE state may need to be archived or analyzed outside of UEDGE. For such exceptions, the Case.dump function can be used to write **all** UEDGE variables to a UETOOLS save-file. This save-file is a complete archive of the UEDGE case, and any UEDGE variable available can be accessed using UETOOLS.

**NOTE:** because UEDGE has ~3700 variables, the on-disk size of a UETOOLS dump for a UEDGE case with 7 ion and 2 gas species on a 106x46 grid is 1.1 Gb. Thus, it is not feasible to create dumps for all files. In contrast, the save-file for the same case, using the default variables written, was 9.2 Mb.